In [1]:
using CSV, DataFrames, Statistics

In [2]:
# prefix = "aistats-rev/"
prefix = "tmlr-rev/"

"tmlr-rev/"

In [3]:
function method_category(meth)
    if startswith(meth, "Imp-then-Reg")
        return "Imp-then-Reg"
    elseif startswith(meth, "Joint Imp-then-Reg")
        return "Joint Imp-then-Reg"
    elseif meth ∈ ["Static", "Affine", "Finite"]
        return "Adaptive LR"
    elseif startswith(meth, "Complete Features")
        return "Complete Features"
    else 
        return meth
    end
end

method_category (generic function with 1 method)

In [4]:
pb_datasets = ["cylinder-bands", "ozone-level-detection-eight", "ozone-level-detection-one", "thyroid-disease-thyroid-0387", "trains",
                "credit-approval", "Ecdat-Mofa", "sleep"]

8-element Vector{String}:
 "cylinder-bands"
 "ozone-level-detection-eight"
 "ozone-level-detection-one"
 "thyroid-disease-thyroid-0387"
 "trains"
 "credit-approval"
 "Ecdat-Mofa"
 "sleep"

## For Real X - Syn Y Experiments

In [5]:
# prefix ="aistats-rev/"
prefix = "tmlr-rev/"

setting = prefix*"fakey/"
for y_model in ["linear", "nn"]
    for m_model = ["mar", "nmar", "mar_adv"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
        
        # filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv")]
        # res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)
        
        # for subdir = ["all/", "itr/", "itr_nn/"]
        filelist = [f for f in readdir(directory*"itr_nn/") if endswith(f, ".csv")]
        res = similar(CSV.read(directory*"itr_nn/"*filelist[1], DataFrame),0)
        
        for subdir = ["itr_nn/"]
            filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv")]
#             res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
            for i in 1:length(filelist)
                res = vcat(res, CSV.read(directory*subdir*filelist[i], DataFrame))
            end
        end

        res[!,:method_cat] = map(t -> method_category(t), res[:,:method])
        res[!,:X_setting] .= "real_X_"*m_model
        res[!,:Y_setting] .= "syn_Y_"*y_model

        CSV.write(directory*"FINAL_results.csv", res)
    end
end

In [6]:
setting = prefix*"fakey/"

for y_model in ["linear", "nn"]
    for m_model = ["mar", "nmar", "mar_adv"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
  
        res = CSV.read(directory*"FINAL_results.csv", DataFrame)
        
        res[!,:method] .= map(t -> (t == "Affine" ? "Adaptive LR - Affine" : t), res[:,:method]) 
        res[!,:method] .= map(t -> (t == "Finite" ? "Adaptive LR - Finite" : t), res[:,:method]) 
        res[!,:method] .= map(t -> (t == "Static" ? "Adaptive LR - Affine intercept only" : t), res[:,:method])
        
        filter!(t -> t[:dataset] ∉ pb_datasets, res)

        for method in ["Oracle X", "Oracle XM", "Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Imp-then-Reg 5", "Joint Imp-then-Reg", "Adaptive LR"]
            aux = filter(t -> startswith(t[:method], method), res)

            idcols = [:dataset, :X_setting, :Y_setting, :SNR, :k, :kMissing, :splitnum]
            gd = groupby(aux, idcols)

            aux = similar(aux, 0)
            for subdf in gd 
                scoremax = argmax(subdf[:,:score])
                push!(aux, subdf[scoremax,names(aux)])
            end
            aux[!,:method] .= method*" - best"

            res = vcat(res, aux)
        end
        
        CSV.write(directory*"FINAL_results.csv", res)
    end
end

Sanity check

In [7]:
res = DataFrames.similar(CSV.read(prefix*"fakey/"*"linear"*"_"*"mar"*"/FINAL_results.csv", DataFrame), 0)
for y_model in ["linear", "nn"]
    for m_model = ["mar", "nmar", "mar_adv"]
        dir = y_model*"_"*m_model*"/"
        directory = prefix*"fakey/"*dir
  
        res = vcat(res, CSV.read(directory*"FINAL_results.csv", DataFrame))
    end
end

In [8]:
gd = groupby(res, [:dataset, :X_setting, :Y_setting, :kMissing, :method])
aggres = combine(gd, nrow)

Row,dataset,X_setting,Y_setting,kMissing,method,nrow
,String,String15,String15,Int64,String31,Int64
1,COUNT-loomis,real_X_mar,syn_Y_linear,0,Imp-then-Reg 1 - nn,10
2,COUNT-loomis,real_X_mar,syn_Y_linear,0,Imp-then-Reg 2 - nn,10
3,COUNT-loomis,real_X_mar,syn_Y_linear,0,Imp-then-Reg 3 - nn,10
4,COUNT-loomis,real_X_mar,syn_Y_linear,0,Imp-then-Reg 4 - nn,10
5,COUNT-loomis,real_X_mar,syn_Y_linear,0,Imp-then-Reg 5 - nn,10
6,Ecdat-MCAS,real_X_mar,syn_Y_linear,0,Imp-then-Reg 1 - nn,10
7,Ecdat-MCAS,real_X_mar,syn_Y_linear,0,Imp-then-Reg 2 - nn,10
8,Ecdat-MCAS,real_X_mar,syn_Y_linear,0,Imp-then-Reg 3 - nn,10
9,Ecdat-MCAS,real_X_mar,syn_Y_linear,0,Imp-then-Reg 4 - nn,10


In [9]:
feasible_combinations = unique(aggres[:,[:dataset, :kMissing]])
feasible_combinations = crossjoin(
                            crossjoin(
                                crossjoin(feasible_combinations, DataFrame(splitnum=1:10)),
                                DataFrame(X_setting = ["real_X_mar", "real_X_nmar", "real_X_mar_adv"])), 
                                DataFrame(Y_setting = ["syn_Y_linear", "syn_Y_nn"])
                            )

pbdatasets = antijoin(feasible_combinations, unique(res[:,[:dataset, :kMissing, :splitnum, :X_setting, :Y_setting, :method]]), on=[:dataset, :kMissing, :splitnum, :X_setting, :Y_setting])
pbdatasets = unique(pbdatasets[:,[:dataset, :X_setting, :Y_setting, :splitnum]])

dataset_list = [d for d in readdir("../datasets/") if !startswith(d, ".")]
sort!(dataset_list)

pbdatasets[!,:array_num] .= (pbdatasets[:,:splitnum] .- 1) .* 71
pbdatasets[!,:array_num] .+= map(t -> findfirst(t .== dataset_list)-1, pbdatasets[:,:dataset])

pbdatasets[!,:back_dnum] .= map(t -> dataset_list[mod(t, 71) + 1], pbdatasets[:,:array_num])
pbdatasets[!,:back_splitnum] .= map(t -> div(t, 71) + 1, pbdatasets[:,:array_num])

@assert all(pbdatasets[:,:back_dnum] .== pbdatasets[:,:dataset])
@assert all(pbdatasets[:,:back_splitnum] .== pbdatasets[:,:splitnum])

In [10]:
function list_to_slurmarray(l)
    s = ""
    begin_seq = -1; last_seq = -1; is_sequence = false

    for an in l
        if an == last_seq + 1
            last_seq = an
        else 
            if is_sequence
                if begin_seq == last_seq
                    s *= string(begin_seq)*","
                else
                    s *= string(begin_seq)*"-"*string(last_seq)*","
                end
            end
            begin_seq = an
            last_seq = an
            is_sequence = true
        end
    end 
    s
end

list_to_slurmarray (generic function with 1 method)

In [11]:
for y_model in ["linear", "nn"]
    println(y_model)
    for m_model = ["mar", "nmar", "mar_adv"]
        println(m_model)
        aux = filter(t -> t[:Y_setting] == "syn_Y_"*y_model && t[:X_setting] == "real_X_"*m_model, pbdatasets)
        l = sort(unique(aux[:,:array_num]))
        @show list_to_slurmarray(l)
    end
    println()
end

linear
mar
list_to_slurmarray(l) = "47,118,166,189,237,256,260,327,331,402,473,544,611,615,"
nmar
list_to_slurmarray(l) = "47,118,256,260,331,402,473,540,544,615,682,686,"
mar_adv
list_to_slurmarray(l) = "47,118,185,189,256,260,331,402,473,544,611,615,"

nn
mar
list_to_slurmarray(l) = "47,114,118,189,256,260,327,331,402,473,611,615,"
nmar
list_to_slurmarray(l) = "47,118,185,189,256,260,331,402,473,544,615,"
mar_adv
list_to_slurmarray(l) = "47,118,189,256,260,331,402,473,544,615,686,"



In [ ]:
47,114,118,166,185,189,237,256,260,327,331,402,473,540,544,611,615,682,686

In [54]:
for y_model in ["linear", "nn"]
    println(y_model)
    for m_model = ["mar", "nmar", "mar_adv"]
        println(m_model)
        aux = filter(t -> t[:Y_setting] == "syn_Y_"*y_model && t[:X_setting] == "real_X_"*m_model, pbdatasets)
        l = sort(unique(aux[:,:array_num]))
        @show list_to_slurmarray(l)
    end
    println()
end

linear
mar
list_to_slurmarray(l) = "49,120,166,185,189,191,237,256,260,262,308,327,331,333,379,398,402,404,450,469,473,475,521,540,544,546,592,611,615,617,663,682,686,"
nmar
list_to_slurmarray(l) = "49,120,165-166,189,191,236-237,260,262,307-308,327,331,333,347,378-379,398,402,404,418,449-450,469,473,475,489,491,520-521,540,544,546,560,562,591-592,611,615,617,631,633,662-663,682,686,688,702,"
mar_adv
list_to_slurmarray(l) = "24,95,120,166,189,191,237,256,260,262,308,327,331,333,379,398,402,404,450,469,473,475,521,540,544,546,592,611,615,617,663,682,686,"

nn
mar
list_to_slurmarray(l) = "49,95,118,120,166,185,189,191,237,256,260,262,308,327,331,333,379,398,402,404,450,469,473,475,521,540,544,546,592,611,615,617,663,682,686,"
nmar
list_to_slurmarray(l) = "49,120,189,191,260,262,327,331,333,398,402,404,469,473,475,540,544,546,592,611,615,617,662-663,682,686,"
mar_adv
list_to_slurmarray(l) = "24,49,67,95,118,120,138,166,189,191,209,237,260,262,280,308,331,333,351,379,398,402,404,422,450,46

## For Real Data Experiments

In [12]:
prefix

"tmlr-rev/"

In [9]:
for directory = prefix .* ["realy/"]
    filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
    res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)
    
    for subdir = ["all/", "itr/", "jitr/", "itr_nn/"]
        filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
#         res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
        for i in 1:length(filelist)
            res = vcat(res, CSV.read(directory*subdir*filelist[i], DataFrame))
        end
    end
    
    # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
    res[!,:method_cat] = map(method_category, res[:,:method])
    res[!,:X_setting] .= "real_X"
    res[!,:Y_setting] .= "real_Y"
    
    CSV.write(directory*"FINAL_results.csv", res)
end

Create `best` variant

In [36]:
res = CSV.read(prefix*"realy/"*"FINAL_results.csv", DataFrame)

filter!(t -> t[:dataset] ∉ pb_datasets, res)

res[!,:method] .= map(t -> (t == "Affine" ? "Adaptive LR - Affine" : t), res[:,:method]) 
res[!,:method] .= map(t -> (t == "Finite" ? "Adaptive LR - Finite" : t), res[:,:method]) 
res[!,:method] .= map(t -> (t == "Static" ? "Adaptive LR - Affine intercept only" : t), res[:,:method])

for method in ["Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Imp-then-Reg 5", "Joint Imp-then-Reg", "Adaptive LR"]
    aux = filter(t -> startswith(t[:method], method), res)
    # @show size(aux)
    idcols = [:dataset, :SNR, :k, :kMissing, :splitnum]
    gd = groupby(aux, idcols)

    aux = similar(aux, 0)
    for subdf in gd 
        scoremax = argmax(subdf[:,:score])
        push!(aux, subdf[scoremax,names(aux)])
    end
    aux[!,:method] .= method*" - best"

    res = vcat(res, aux)
end

CSV.write(prefix*"realy/"*"FINAL_results.csv", res)

"aistats-rev/realy/FINAL_results.csv"

Sanity check

In [10]:
res = CSV.read(prefix*"realy/"*"FINAL_results.csv", DataFrame)
unique(filter( t-> t[:nrow] < 10, combine(groupby(res, [:dataset, :method]), nrow))[:,[:dataset, :nrow]])

Row,dataset,nrow
,String,Int64
1,mlmRev-star,6
2,pscl-politicalInformation,3


In [16]:
feasible_combinations = unique(res[:,[:dataset]])
feasible_combinations = crossjoin(feasible_combinations, DataFrame(splitnum=1:10))

pbdatasets = leftjoin(feasible_combinations, res, on=[:dataset, :splitnum])
gd = groupby(pbdatasets, [:dataset, :splitnum])
sort(combine(gd, nrow), :nrow)

# unique(res[:,[:dataset, :kMissing, :splitnum, :X_setting, :Y_setting]]), on=[:dataset, :kMissing, :splitnum, :X_setting, :Y_setting])
# pbdatasets = unique(pbdatasets[:,[:dataset, :X_setting, :Y_setting, :splitnum]])

# dataset_list = [d for d in readdir("../datasets/") if !startswith(d, ".")]
# sort!(dataset_list)

# pbdatasets[!,:array_num] .= (pbdatasets[:,:splitnum] .- 1) .* 71
# pbdatasets[!,:array_num] .+= map(t -> findfirst(t .== dataset_list)-1, pbdatasets[:,:dataset])

# pbdatasets[!,:back_dnum] .= map(t -> dataset_list[mod(t, 71) + 1], pbdatasets[:,:array_num])
# pbdatasets[!,:back_splitnum] .= map(t -> div(t, 71) + 1, pbdatasets[:,:array_num])

# @assert all(pbdatasets[:,:back_dnum] .== pbdatasets[:,:dataset])
# @assert all(pbdatasets[:,:back_splitnum] .== pbdatasets[:,:splitnum])

Row,dataset,splitnum,nrow
,String,Int64,Int64
1,COUNT-loomis,1,41
2,COUNT-loomis,2,41
3,COUNT-loomis,3,41
4,COUNT-loomis,4,41
5,COUNT-loomis,5,41
6,COUNT-loomis,6,41
7,COUNT-loomis,7,41
8,COUNT-loomis,8,41
9,COUNT-loomis,9,41


In [12]:
dataset_list = [d for d in readdir("../datasets/") if !startswith(d, ".")]
sort!(dataset_list)

71-element Vector{String}:
 "COUNT-loomis"
 "Ecdat-MCAS"
 "Ecdat-Males"
 "Ecdat-Mofa"
 "Ecdat-RetSchool"
 "Ecdat-Schooling"
 "MASS-Cars93"
 "MASS-Pima.tr2"
 "MASS-survey"
 "Zelig-coalition2"
 ⋮
 "thyroid-disease-allhyper"
 "thyroid-disease-allhypo"
 "thyroid-disease-allrep"
 "thyroid-disease-dis"
 "thyroid-disease-sick"
 "thyroid-disease-sick-euthyroid"
 "thyroid-disease-thyroid-0387"
 "trains"
 "wiki4he"

In [13]:
unique(filter(t -> t[:nrow] < 10, combine(groupby(filter(t -> !startswith(t[:method], "Imp-then-Reg 4 - linear"), res), [:dataset, :method]), nrow))[:,:dataset])

2-element Vector{String}:
 "mlmRev-star"
 "pscl-politicalInformation"

In [14]:
unfinished_datasets = filter(t -> t[:nrow] < 10, 
    combine(
        groupby(    combine(groupby(filter(t -> !startswith(t[:method], "Imp-then-Reg 4 - linear"), res), [:dataset, :method]), nrow), 
                    [:dataset]), 
        :nrow => minimum => :nrow))[:,:dataset]
sort([findfirst(i .== dataset_list)-1 for i in unfinished_datasets])

2-element Vector{Int64}:
 43
 49

In [15]:
unfinished_datasets = filter(t -> t[:nrow] < 10, 
    combine(
        groupby(    combine(groupby(res, [:dataset, :method]), nrow), 
                    [:dataset]), 
        :nrow => minimum => :nrow))[:,:dataset]
sort([findfirst(i .== dataset_list)-1 for i in unfinished_datasets])

2-element Vector{Int64}:
 43
 49

In [21]:
unfinished_datasets = filter(t -> t[:nrow] < 10, 
    combine(
        groupby(    combine(groupby(filter(t -> !startswith(t[:method], "Imp-then-Reg"), res), [:dataset, :method]), nrow), 
                    [:dataset]), 
        :nrow => minimum => :nrow))[:,:dataset]
sort([findfirst(i .== dataset_list)-1 for i in unfinished_datasets])

Any[]

In [41]:
sort(union([71*i + (17) for i in 0:9], [71*i + 49 for i in 0:9]))

20-element Vector{Int64}:
  43
  49
 114
 120
 185
 191
 256
 262
 327
 333
 398
 404
 469
 475
 540
 546
 611
 617
 682
 688

In [40]:
array_num = 545
@show d_num = mod(array_num, 71) + 1
@show iter_do = div(array_num, 71) + 1

d_num = mod(array_num, 71) + 1 = 49
iter_do = div(array_num, 71) + 1 = 8


8

# For Synthetic-Data Experiments

In [30]:
prefix

"aistats-rev/"

In [31]:
setting = prefix*"synthetic_discrete/"
for y_model in ["linear", "nn"]
    for m_model = ["mar", "censoring"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir

        # filelist = [f for f in readdir(directory*"itr_nn/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
        # res = similar(CSV.read(directory*"itr_nn/"*filelist[1], DataFrame),0)

        # for subdir = ["itr_nn/"]

        filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
        res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)

        for subdir = ["all/", "itr/", "itr_nn/"]
            filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
            # @show filelist

            for i in 1:length(filelist)
                aux = CSV.read(directory*subdir*filelist[i], DataFrame)
                try
                    res = vcat(res, aux)
                catch 
                    println("Error with ", directory*subdir*filelist[i])
                end
            end
        end

        res[!,:method] .= map(t -> replace(t, "Encoding as new category" => "Imp-then-Reg 4"), res[:,:method])
        res[!,:method] .= map(t -> replace(t, "Mode impute" => "Imp-then-Reg 5"), res[:,:method])

        # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
        res[!,:method_cat] = map(method_category, res[:,:method])
        res[!,:X_setting] .= "syn_X_"*m_model
        res[!,:Y_setting] .= "syn_Y_"*y_model

        for method in ["Imp-then-Reg 4", "Imp-then-Reg 5"]
            aux = filter(t -> startswith(t[:method], method), res)
            # @show nrow(aux)
            idcols = [:dataset, :X_setting, :Y_setting, :SNR, :k, :pMissing, :splitnum]
            gd = groupby(aux, idcols)

            aux = similar(aux, 0)
            for subdf in gd 
                scoremax = argmax(subdf[:,:score])
                # @show subdf[scoremax,names(aux)]
                push!(aux, subdf[scoremax,names(aux)])
            end
            aux[!,:method] .= method*" - best"

            res = vcat(res, aux)
        end

        CSV.write(directory*"FINAL_results.csv", res)   
    end
end

In [42]:
prefix

"aistats-rev/"

In [43]:
setting = prefix*"synthetic/"
files_with_issues = []
# for y_model in ["linear", "tree", "nn"]
for y_model in ["linear", "nn"]
    for m_model = ["mar", "censoring"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
        # filelist = [f for f in readdir(directory*"itr_nn/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
        
        # res = similar(CSV.read(directory*"itr_nn/"*filelist[1], DataFrame),0)
        # for subdir = ["itr_nn/"]

        filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
        
        res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)
        for subdir = ["all/","itr_nn/"]
            filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]

            for i in 1:length(filelist)
                try
                    aux = CSV.read(directory*subdir*filelist[i], DataFrame)
                    if any(aux[:,:pMissing] .> 0)
                        missingproba = unique(aux[aux[:,:pMissing] .> 0,:pMissing])[1]
                        aux[!,:pMissing] .= missingproba
                    end
                    res = vcat(res, aux)
                catch 
                    println("Error with ", directory*subdir*filelist[i])
                    push!(files_with_issues, directory*subdir*filelist[i])
                end
            end
        end

        res[!,:method_cat] = map(method_category, res[:,:method])
        res[!,:X_setting] .= "syn_X_"*m_model
        res[!,:Y_setting] .= "syn_Y_"*y_model

        for method in ["Oracle X", "Oracle XM", "Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Imp-then-Reg 5", "Joint Imp-then-Reg", "Adaptive LR"]
            aux = filter(t -> startswith(t[:method], method), res)

            idcols = [:dataset, :X_setting, :Y_setting, :SNR, :k, :pMissing, :splitnum]
            gd = groupby(aux, idcols)

            aux = similar(aux, 0)
            for subdf in gd 
                scoremax = argmax(subdf[:,:score])
                push!(aux, subdf[scoremax,names(aux)])
            end
            aux[!,:method] .= method*" - best"

            res = vcat(res, aux)
        end
        
        CSV.write(directory*"FINAL_results.csv", res)   
    end
end

In [44]:
1+1

2

Sanity check

In [20]:
df = CSV.read(prefix*"synthetic/linear_mar/FINAL_results.csv", DataFrame) 
df[!,:setting] .= "1 - Lin-MAR"

aux = CSV.read(prefix*"synthetic/linear_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "2 - Lin-NMAR"
df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic/nn_mar/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "5 - NN-MAR"
df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic/nn_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "6 - NN-NMAR"
df = vcat(df, aux)

;

In [ ]:
df = CSV.read(prefix*"synthetic_discrete/linear_mar/FINAL_results.csv", DataFrame) 
df[!,:setting] .= "1 - Lin-MAR"

aux = CSV.read(prefix*"synthetic_discrete/linear_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "2 - Lin-NMAR"
df = vcat(df, aux)
 
# aux = CSV.read(prefix*"synthetic_discrete/tree_mar/FINAL_results.csv", DataFrame) 
# aux[!,:setting] .= "3 - Tree-MAR"
# df = vcat(df, aux)

# aux = CSV.read(prefix*"synthetic_discrete/tree_censoring/FINAL_results.csv", DataFrame) 
# aux[!,:setting] .= "4 - Tree-NMAR"
# df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic_discrete/nn_mar/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "5 - NN-MAR"
df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic_discrete/nn_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "6 - NN-NMAR"
df = vcat(df, aux)

In [21]:
combine(groupby(df, [:dataset, :method, :setting]), nrow)

Row,dataset,method,setting,nrow
,String15,String31,String,Int64
1,n_1000_p_10,Imp-then-Reg 1 - nn,1 - Lin-MAR,80
2,n_1000_p_10,Imp-then-Reg 2 - nn,1 - Lin-MAR,80
3,n_1000_p_10,Imp-then-Reg 3 - nn,1 - Lin-MAR,80
4,n_1000_p_10,Imp-then-Reg 4 - nn,1 - Lin-MAR,80
5,n_100_p_10,Imp-then-Reg 1 - nn,1 - Lin-MAR,80
6,n_100_p_10,Imp-then-Reg 2 - nn,1 - Lin-MAR,80
7,n_100_p_10,Imp-then-Reg 3 - nn,1 - Lin-MAR,80
8,n_100_p_10,Imp-then-Reg 4 - nn,1 - Lin-MAR,80
9,n_120_p_10,Imp-then-Reg 1 - nn,1 - Lin-MAR,80


In [22]:
unique(combine(groupby(df, [:dataset, :method, :setting]), nrow)[:,:nrow])

2-element Vector{Int64}:
 80
 70

In [25]:
df

Row,dataset,SNR,k,pMissing,splitnum,method,r2,osr2,r2list,osr2list,muvec,time,hp,score,method_cat,X_setting,Y_setting,setting
,String15,Int64,Int64,Float64,Int64,String31,Float64,Float64,String,String,String,Float64,String31,Float64,String15,String15,String15,String
1,n_1000_p_10,2,5,0.1,1,Imp-then-Reg 1 - nn,0.783002,0.751881,"Any[0.7665171843286452, 0.7893688177528002, 0.817890224476437, 0.8036376354991142, 0.8596366435831156, 0.6794559329720173, 0.8255995802632057, 0.7932532468734835, 0.6801241320648609, 0.7697311410179672, 0.7853428465486968, -0.14125156335652678, -2.6213999614988786, 0.7836464677029975, 0.48283759691039496, 0.6985706718146727, 0.8828181855263525, 0.613880113666227, 0.8529106628148347, 0.26800333636336826, 0.4312812175230756, 0.32808287495344757, 0.7954332113661159, 0.6461918753114773, 0.8413415048750846, 0.7482858639789494, 0.45331449723493056, -0.319303024917974, 0.9435875483957107, 0.609444513131788, 0.8375865206899261, -0.4003122840987927, 0.36382307143911496, -0.09835751708094853, 0.7673083877612625, 0.8710774660429738, 0.8497286891177294, 0.5302368964187063, 0.5509431808908263, 0.9357898844478688, 0.6173602852352, -Inf, 0.11734489013879679, 0.5492124607929425, 0.8774381044892472, 0.4961494381574233, 0.26043816106758044, 0.9318871916838743, 0.8340444739105703, 0.6950496067109759, 0.6996386744453672, -4.068040894228982, 0.7487812165423975, 0.9192184101213814, 0.9103397566045368, -Inf, 0.868464490317837, -Inf, -26.194196960340797, 0.9879315708713736, -Inf, -Inf, -Inf, -Inf, -Inf, 0.9562525904598034, -Inf, -Inf, -Inf, 0.36146436668297743, -Inf, -Inf, -Inf, -Inf, 0.03581616526594078, -Inf, -Inf, -6.706069592621754, 0.8791090182581316, -Inf, -Inf, -7.353232277921936, -Inf, -Inf, -Inf, -Inf, 0.8766534149488281, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, 0.11915598568978736, -Inf, 0.7329947531807602]","Any[0.7533434727434916, 0.8037345427780918, 0.801091541948193, 0.8007502185610421, 0.7593401249053844, 0.7953275600860703, 0.7104046230222195, 0.7896755196027223, 0.7976046380948352, 0.7348855590308163, 0.7843037337786878, 0.7762867554262245, 0.2606144580701999, 0.8207606392407211, 0.732780895226183, 0.8076320574830487, 0.9073364799484116, 0.6911663494068845, 0.6719979652951384, 0.8859652509898766, 0.7328795072402021, 0.7980052642660229, 0.8934376901623409, 0.802096588017864, 0.8701239974730035, 0.8478849214126659, 0.583579958875988, 0.9352982485171242, 0.7364575044174088, 0.7263647011033944, 0.7243856002901728, 0.8471031669051663, 0.9088612467233923, 0.8229505915793724, 0.8937747827013545, 0.8133201220031822, 0.8659186494466297, 0.6486402106480877, 0.6397369309060714, 0.7240070603104105, 0.8886599489996887, 0.8432553668484993, 0.6813032572836998, 0.7443056106764531, 0.7315750611175126, 0.48032726104171986, 0.7905149315000028, 0.6810310992022784, 0.9176607783529119, 0.8507442460822956, 0.7665969506205166, 0.8769600648907918, 0.8433764792654727, 0.6148707254401669, 0.9069935448001605, 0.7167954529733869, 0.7420524532711759, 0.9567343061692194, 0.505121207810054, 0.81418855777992, 0.8728106686162509, 0.6903808182616198, 0.9759403767751433, 0.9898534388495667, 0.320505167926451, 0.974180438883839, 0.27748718852662624, 0.9583687821634513, 0.9248942225551539, 0.8922518436649951, 0.9905967953608596, 0.9614222082125097, 0.5592893556705427, 0.9333689186420774, NaN, 0.9871552174054466, 0.46362290119727223, -55.52448204205184, 0.9675299011821044, 0.9607117609873352, 0.9523900151543707, -405.12377523098894, 0.8132824812298787, 0.34888789864703895, 0.6052839845803966, 0.8796903402451017, NaN, -0.6092324821737163, 0.9969813443984485, -0.9926120429373997, 0.9212446959641847, 0.9919953863878542, 0.9954529716678489, 0.9994265414934197, NaN, -2.260342809991455, 0.7307303641950899]",Any[],104.997,Dict(:hidden_nodes => 5),0.771493,Imp-then-Reg,syn_X_mar,syn_Y_linear,1 - Lin-MAR
2,n_1000_p_10,2,5,0.1,1,Imp-then-Reg 2 - nn,0.782897,0.754546,"Any[0.7695939310834479, 0.7958369386061889, 0.827814557522069, 0.8124808499360384, 0.854429

In [28]:
unique(filter(t -> t[:dataset] == "n_20_p_10", df)[:,:pMissing])

7-element Vector{Float64}:
 0.1
 0.2
 0.3
 0.4
 0.5
 0.6
 0.7

In [24]:
filter( t-> t[:nrow] < 80, combine(groupby(df, [:dataset, :method, :setting]), nrow))

Row,dataset,method,setting,nrow
,String15,String31,String,Int64
1,n_20_p_10,Imp-then-Reg 1 - nn,1 - Lin-MAR,70
2,n_20_p_10,Imp-then-Reg 2 - nn,1 - Lin-MAR,70
3,n_20_p_10,Imp-then-Reg 3 - nn,1 - Lin-MAR,70
4,n_20_p_10,Imp-then-Reg 4 - nn,1 - Lin-MAR,70
5,n_20_p_10,Imp-then-Reg 1 - best,1 - Lin-MAR,70
6,n_20_p_10,Imp-then-Reg 2 - best,1 - Lin-MAR,70
7,n_20_p_10,Imp-then-Reg 3 - best,1 - Lin-MAR,70
8,n_20_p_10,Imp-then-Reg 4 - best,1 - Lin-MAR,70
9,n_20_p_10,Imp-then-Reg 1 - nn,2 - Lin-NMAR,70


In [ ]:
pbdata = unique(filter( t -> t[:nrow] < 90, combine(groupby(df, [:dataset, :method, :setting]), nrow))[:,[:method, :setting, :dataset]])

In [ ]:
aux = filter( t -> t[:dataset] ∈ pbdata && t[:nrow] < 10, combine(groupby(df, [:dataset, :method, :setting, :pMissing]), nrow))
unique(aux[:,[:dataset, :setting, :pMissing, :nrow]])

In [ ]:
filter(t -> t[:pMissing] == 0.1, unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))

In [ ]:
filter(t -> t[:pMissing] == 0.3, unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))

In [ ]:
filter(t -> startswith(t[:setting], "5"), unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))